# **KAGGLE PLAYGROUND SERIES S5E1**

todo:
- ensembling strategie überarbeiten
- feature engineering und autofe

In [1]:
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from stacking_ensemble import StackingEnsemble

train = pd.read_csv('train.csv', index_col='id')
test = pd.read_csv('test.csv', index_col='id')
test_indices = test.index
label = 'num_sold'
time_limit = 24*60*60
n_splits = 4

# **Preprocessing**

In [2]:
def preprocess(df, is_train):
    df['date'] = pd.to_datetime(df['date'])
    df['country'] = df['country'].astype('category')
    df['store'] = df['store'].astype('category')
    df['product'] = df['product'].astype('category')
    if is_train:
        df[label] = df.groupby(['country', 'store', 'product'])[label].transform(lambda x: x.fillna(x.median()))
        df[label] = df.groupby(['country', 'product'])[label].transform(lambda x: x.fillna(x.median()))
        df[label] = df[label].astype('float32')
    return df

train = preprocess(train, is_train=True)
test = preprocess(test, is_train=False)

/var/folders/r7/1qf9bs6920bgntn0wmtsp71m0000gn/T/ipykernel_32877/3684526116.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[label] = df.groupby(['country', 'store', 'product'])[label].transform(lambda x: x.fillna(x.median()))
/var/folders/r7/1qf9bs6920bgntn0wmtsp71m0000gn/T/ipykernel_32877/3684526116.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[label] = df.groupby(['country', 'product'])[label].transform(lambda x: x.fillna(x.median()))


# **Stacked Ensemble**

In [3]:
def make_autogluon_model(time_limit):
    from autogluon.tabular import TabularPredictor
    
    class AutoGluonWrapper:
        def __init__(self, time_limit):
            self.model = None
            self.time_limit = time_limit
        
        def fit(self, train_data, label):
            self.model = TabularPredictor(
                label=label,
                eval_metric='mean_absolute_percentage_error'
            )
            self.model.fit(
                train_data,
                presets='best_quality',
                time_limit=self.time_limit,
            )
            
        def predict(self, test_data):
            return self.model.predict(test_data).values
    
    return AutoGluonWrapper(time_limit=time_limit)

def make_mljar_model(time_limit):
    from supervised.automl import AutoML
    class MLJARWrapper:
        def __init__(self, time_limit):
            self.model = None
            self.time_limit = time_limit
        
        def fit(self, train_data, label):
            self.model = AutoML(
                eval_metric='mape',
                mode='Compete',
                total_time_limit=self.time_limit
            )
            self.model.fit(
                train_data.drop(columns=[label]),
                train_data[label]
            )
        
        def predict(self, test_data):
            return self.model.predict(test_data)
    
    return MLJARWrapper(time_limit=time_limit)

def make_flaml_model(time_limit):
    from flaml import AutoML
    
    class FLAMLWrapper:
        def __init__(self, time_limit):
            self.model = AutoML()
            self.time_limit = time_limit
        
        def fit(self, train_data, label):
            self.model.fit(
                dataframe=train_data,
                label=label,
                time_budget=self.time_limit,
                metric='mape',
                task='regression'
            )
        
        def predict(self, test_data):
            return self.model.predict(test_data)
    
    return FLAMLWrapper(time_limit=time_limit)

# Our base models are these "factory" functions:
base_model_factories = [
    make_autogluon_model,
    make_mljar_model,
    make_flaml_model
]

# Our meta-model: a simple XGBRegressor
meta_model = LinearRegression()

# Instantiate the stacking ensemble
stacker = StackingEnsemble(
    base_models=base_model_factories,
    meta_model=meta_model,
    n_splits=n_splits,
    random_state=42,
    shuffle=True,
    oof_total_training_time = time_limit
)

# Fit the stacker (this will do the K-fold OOF, train meta-model, then final base-models)
stacker.fit(train, label)

predictions = stacker.predict(test)
print("Stacked predictions:", predictions)

No path specified. Models will be saved in: "AutogluonModels/ag-20250106_094001"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.2
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.1.0: Thu Oct 10 21:03:15 PDT 2024; root:xnu-11215.41.3~2/RELEASE_ARM64_T6000
CPU Count:          8
Memory Avail:       8.25 GB / 16.00 GB (51.6%)
Disk Space Avail:   162.74 GB / 460.43 GB (35.3%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_s

(_ray_fit pid=32910) [1000]	valid_set's l2: 4851.26	valid_set's mean_absolute_percentage_error: -0.119099
(_ray_fit pid=32910) [2000]	valid_set's l2: 4522.54	valid_set's mean_absolute_percentage_error: -0.106096 [repeated 8x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_ray_fit pid=32910) [3000]	valid_set's l2: 4392.62	valid_set's mean_absolute_percentage_error: -0.101132 [repeated 8x across cluster]
(_ray_fit pid=32909) [4000]	valid_set's l2: 4491.67	valid_set's mean_absolute_percentage_error: -0.101771 [repeated 8x across cluster]
(_ray_fit pid=32916) [4000]	valid_set's l2: 4340.13	valid_set's mean_absolute_percentage_error: -0.101996 [repeated 3x across cluster]
(_ray_fit pid=32916) [5000]	valid_set's l2: 4295.09	valid_set's mean_absolute_percentage_error: -0.100827 [repeated 8x across cluster]
(_ray_

(_dystack pid=32900) 	-0.0985	 = Validation score   (-mean_absolute_percentage_error)
(_dystack pid=32900) 	62.91s	 = Training   runtime
(_dystack pid=32900) 	133.16s	 = Validation runtime
(_dystack pid=32900) Fitting model: LightGBM_BAG_L1 ... Training model for up to 1117.06s of the 1716.28s of remaining time.
(_dystack pid=32900) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.64%)


(_ray_fit pid=32938) [1000]	valid_set's l2: 2918.43	valid_set's mean_absolute_percentage_error: -0.11624
(_ray_fit pid=32937) [1000]	valid_set's l2: 3098.99	valid_set's mean_absolute_percentage_error: -0.123709
(_ray_fit pid=32933) [2000]	valid_set's l2: 2879.89	valid_set's mean_absolute_percentage_error: -0.104677 [repeated 10x across cluster]
(_ray_fit pid=32935) [3000]	valid_set's l2: 3010.78	valid_set's mean_absolute_percentage_error: -0.0962786 [repeated 8x across cluster]
(_ray_fit pid=32934) [5000]	valid_set's l2: 2882.35	valid_set's mean_absolute_percentage_error: -0.0943345 [repeated 9x across cluster]


(_dystack pid=32900) 	-0.0939	 = Validation score   (-mean_absolute_percentage_error)
(_dystack pid=32900) 	28.29s	 = Training   runtime
(_dystack pid=32900) 	26.78s	 = Validation runtime
(_dystack pid=32900) Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 1084.25s of the 1683.47s of remaining time.
(_dystack pid=32900) 	-0.0652	 = Validation score   (-mean_absolute_percentage_error)
(_dystack pid=32900) 	11.93s	 = Training   runtime
(_dystack pid=32900) 	3.15s	 = Validation runtime
(_dystack pid=32900) Fitting model: CatBoost_BAG_L1 ... Training model for up to 1068.94s of the 1668.16s of remaining time.
(_dystack pid=32900) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.70%)
(_dystack pid=32900) 	-0.1109	 = Validation score   (-mean_absolute_percentage_error)
(_dystack pid=32900) 	760.94s	 = Training   runtime
(_dystack pid=32900) 	0.95s	 = Validation runtime
(_dystack pid=32900) Fitti

...

Linear algorithm was disabled.
AutoML directory: AutoML_5
The task is regression with evaluation metric mape
AutoML will use algorithms: ['Decision Tree', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'mix_encoding', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree mape 0.499468 trained in 1.38 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle
* Step simple_algorithms will try to check up to 3 models
1_DecisionTree mape 0.501645 trained in 11.66 seconds
2_DecisionTree mape 0.332782 trained in 11.55 seconds
3_DecisionTree mape 0.3

2025-01-07 14:16:25,098 supervised.exceptions ERROR KMeans Features not created. No continous features.
2025-01-07 14:16:25,116 supervised.exceptions ERROR KMeans Features not created. No continous features.
2025-01-07 14:16:25,133 supervised.exceptions ERROR KMeans Features not created. No continous features.


Skip mix_encoding because of the time limit.
Skip golden_features because of the time limit.
* Step kmeans_features will try to check up to 3 models
There was an error during 4_Default_LightGBM_KMeansFeatures training.
Please check AutoML_5/errors.md for details.
There was an error during 19_LightGBM_KMeansFeatures training.
Please check AutoML_5/errors.md for details.
There was an error during 20_LightGBM_KMeansFeatures training.
Please check AutoML_5/errors.md for details.
* Step insert_random_feature will try to check up to 1 model
4_Default_LightGBM_RandomFeature mape 0.119393 trained in 302.16 seconds
Drop features ['random_feature']
Skip features_selection because no parameters were generated.
* Step hill_climbing_1 will try to check up to 28 models
56_CatBoost mape 0.048813 trained in 821.92 seconds
57_CatBoost mape 0.04639 trained in 879.8 seconds
Skip hill_climbing_2 because of the time limit.
* Step boost_on_errors will try to check up to 1 model
4_Default_LightGBM_BoostOnErr

2025-01-07 15:10:31,981 flaml.tune.searcher.blendsearch INFO No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


[flaml.automl.logger: 01-07 15:10:32] {2442} INFO -  at 0.7s,	estimator sgd's best error=8.7030,	best estimator lgbm's best error=5.2671
[flaml.automl.logger: 01-07 15:10:32] {2258} INFO - iteration 4, current learner lgbm
[flaml.automl.logger: 01-07 15:10:32] {2442} INFO -  at 0.8s,	estimator lgbm's best error=1.7285,	best estimator lgbm's best error=1.7285
[flaml.automl.logger: 01-07 15:10:32] {2258} INFO - iteration 5, current learner xgboost
[flaml.automl.logger: 01-07 15:10:32] {2442} INFO -  at 0.9s,	estimator xgboost's best error=8.0527,	best estimator lgbm's best error=1.7285
[flaml.automl.logger: 01-07 15:10:32] {2258} INFO - iteration 6, current learner extra_tree
[flaml.automl.logger: 01-07 15:10:32] {2442} INFO -  at 1.0s,	estimator extra_tree's best error=6.1116,	best estimator lgbm's best error=1.7285
[flaml.automl.logger: 01-07 15:10:32] {2258} INFO - iteration 7, current learner rf
[flaml.automl.logger: 01-07 15:10:32] {2442} INFO -  at 1.0s,	estimator rf's best error=1

In [4]:
# MAPE SCORE
score = stacker.score(
    train.drop(columns=[label]),
    train[label],
    mean_absolute_percentage_error
)
print(score)

# CREATE SUBMISSION CSV
pd.DataFrame(
    predictions,
    index=test_indices,
    columns=['num_sold']
).astype(
    {'num_sold':'int32'}
).to_csv('oof_ensemble_linreg_24hrs.csv')

0.04745673527713729


# **BENCHMARKS**

### AutoGluon

In [ ]:
from autogluon.tabular import TabularPredictor

autogluon_model = TabularPredictor(
    label=label,
    eval_metric='mean_absolute_percentage_error'
)
autogluon_model.fit(
    train,
    presets='best_quality',
    time_limit=time_limit,
)
preds['autogluon'] = autogluon_model.predict(train.drop(columns=[label])).values

### MLJar

In [ ]:
from supervised.automl import AutoML

mljar_model = AutoML(
    eval_metric='mape',
    mode='Compete',
    total_time_limit=time_limit
)
mljar_model.fit(
    train.drop(columns=[label]),
    train[label]
)
preds['mljar'] = mljar_model.predict(train.drop(columns=[label]))

### FLAML

In [ ]:
from flaml import AutoML
flaml_model = AutoML()
flaml_model.fit(
    dataframe=train,
    label=label,
    time_budget=time_limit,
    metric='mape',
    task='regression'
)
preds['flaml'] = flaml_model.predict(train.drop(columns=[label]))

### MAPE Scores

In [ ]:
results = pd.merge(
    left=train[[label]],
    right=pd.DataFrame(preds, index=train.index),
    on='id'
)
print(f'autogluon mape: {mean_absolute_percentage_error(results['num_sold'], results['autogluon'])}')
print(f'mljar mape: {mean_absolute_percentage_error(results['num_sold'], results['mljar'])}')
print(f'flaml mape: {mean_absolute_percentage_error(results['num_sold'], results['flaml'])}')
"""
print(f'h2o mape: {mean_absolute_percentage_error(results['num_sold'], results['h2o'])}')
print(f'lama mape: {mean_absolute_percentage_error(results['num_sold'], results['lama'])}')
"""

### Benchmark Submissions

In [11]:
# AUTOGLUON BENCHMARK
autogluon_benchmark_preds = autogluon_model.predict(test)
pd.DataFrame(autogluon_benchmark_preds).astype({'num_sold':'int32'}).to_csv('benchmark_autogluon_2hrs.csv')

# MLJAR BENCHMARK
tmp = test.index
mljar_benchmark_preds = mljar_model.predict(test)
test.index = tmp
pd.DataFrame(mljar_benchmark_preds, index=test.index, columns=['num_sold']).astype({'num_sold':'int32'}).to_csv('benchmark_mljar_2hrs.csv')

# FLAML BENCHMARK
flaml_benchmark_preds = flaml_model.predict(test)
pd.DataFrame(flaml_benchmark_preds, index=test.index, columns=['num_sold']).astype({'num_sold':'int32'}).to_csv('benchmark_flaml_2hrs.csv')

# EQUAL WEIGHTS ENSEMBLE BENCHMARK
pd.DataFrame((autogluon_benchmark_preds + mljar_benchmark_preds + flaml_benchmark_preds) / 3).astype({'num_sold':'int32'}).to_csv('benchmark_ensemble_2hrs.csv')